In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.append('../')

import sqlite3
import json
from backtester import *
from fxcmtoken import major_forex_pairs
# from tqdm.notebook import tqdm

In [2]:
def get_data(ticker, freq, high_param, train_length=500, test_size=0.05):
    conn = sqlite3.connect(f'../PriceData/PriceData_{freq}.db')
    data = pd.read_sql(f"SELECT * FROM '{ticker}'", conn, parse_dates=['date'], index_col=['date'])
    conn.close()
    columns = ['open', 'high', 'low', 'close']
    for col in columns:
        data[col] = data[['bid'+col, 'ask'+col]].mean(axis=1)
#     train_data, test_data = train_test_split(data, test_size=test_size, shuffle=False)
    split_len = int(len(data) * test_size)
    test_len = split_len + high_param
    test_data = data[-test_len:]
    train_data = data[-(train_length+split_len+high_param):-split_len]
    
    return train_data, test_data

def get_signals(ticker, data, param_dict):
    params = param_dict['ThreeInds'][ticker]
    # get indicators
    ema_window = params[0]
    rsi_window = params[1]
    bb_window = params[2]
    
    data['ema'] = talib.EMA(data.close, ema_window)
    data['rsi'] = talib.RSI(data.close, rsi_window)
    data['bb_upper'], data['bb_mid'], data['bb_lower'] = talib.BBANDS(data.close, bb_window)
    data.dropna(axis=0, inplace=True)

    data['ema_signal'] = np.where(data.close > data.ema, 1, -1)
    
    data['rsi_signal'] = np.where(data.rsi > 70, -1, 
                                  np.where(data.rsi < 30, 1, np.nan))
    data['rsi_less_50'] = np.where(data.rsi < 50, 1, -1)
    data['rsi_signal'][(data.rsi_less_50 * data.rsi_less_50.shift(1) < 0)] = 0
    data.rsi_signal.ffill(inplace=True)
    data.rsi_signal.fillna(0, inplace=True)
    
    data['bb_signal'] = np.where(data.close > data.bb_upper, -1, 
                                 np.where(data.close < data.bb_lower, 1, np.nan))
    data['bb_less_mid'] = np.where(data.close < data.bb_mid, 1, -1)
    data['bb_less_mid'][(data.bb_less_mid * data.bb_less_mid.shift(1) < 0)] = 0
    data.bb_signal.ffill(inplace=True)
    data.bb_signal.fillna(0, inplace=True)
    
    signal_cols = ['ema_signal', 'rsi_signal', 'bb_signal']
    signals = data.loc[:,signal_cols].mode(axis=1).iloc[:, 0]
    
    return signals

In [3]:
freq = 'H1'
fname = f'../ta_optimize/json/OPTIMIZE_params_{freq}_1000_100.json'
param_max = int(fname.split('_')[-2])

with open(fname, 'r') as f:
    parameters = json.load(f)
parameters

train_rets = []
test_rets = []

for ticker in tqdm(major_forex_pairs, leave=False):
    train, test = get_data(ticker, freq, max(parameters['ThreeInds'][ticker]))
    train_signals = get_signals(ticker, train, parameters)
    test_signals = get_signals(ticker, test, parameters)
    
    train_backtester = IterativeBacktester(data=train, freq=freq, signals=train_signals)
    train_backtester.backtest(progress_bar=False)
    train_ret = train_backtester.return_df.loc['TotalReturn', 'Portfolio']
    train_rets.append(train_ret)
    
    test_backtester = IterativeBacktester(data=test, freq=freq, signals=test_signals)
    test_backtester.backtest(progress_bar=False)
    test_ret = test_backtester.return_df.loc['TotalReturn', 'Portfolio']
    test_rets.append(test_ret)

In [4]:
parameters

{'ThreeInds': {'AUD/USD': [200, 30, 700],
  'EUR/USD': [100, 70, 1000],
  'GBP/USD': [200, 40, 200],
  'NZD/USD': [700, 40, 900],
  'USD/CAD': [200, 10, 400],
  'USD/CHF': [300, 30, 500],
  'USD/JPY': [100, 10, 500]}}

In [5]:
print(freq)
print('***** Returns *****')
for ticker, train_ret, test_ret in zip(major_forex_pairs, train_rets, test_rets):
    print(f"{ticker} | Train: {train_ret*100:.2f}% | Test: {test_ret*100:.2f}%")
print('-'*50)
print(f"Total | Train: {np.mean(train_rets)*100:.2f}% | Test: {np.mean(test_rets)*100:.2f}%")

H1
***** Returns *****
EUR/USD | Train: 4.96% | Test: 2.80%
GBP/USD | Train: 2.92% | Test: 3.28%
USD/JPY | Train: 1.86% | Test: 0.19%
USD/CHF | Train: 2.85% | Test: 0.07%
USD/CAD | Train: 3.68% | Test: 0.86%
AUD/USD | Train: 5.55% | Test: -0.21%
NZD/USD | Train: 3.49% | Test: 0.25%
--------------------------------------------------
Total | Train: 3.62% | Test: 1.03%
